## Imports


In [ ]:
import json
import os
import textwrap
from time import gmtime, strftime
from typing import Dict, List

import plantuml
from IPython.display import Image, display
from ipywidgets import fixed, interact, widgets
from q2d.checker import convert_str_graph_with_check
from q2d.graph_to_plantuml import GraphConvertConfig

## Data


In [ ]:
class Config:
    input_dataset = "./datasets/claude_sonnet_synth.json"
    previous_data = "./datasets/claude_sonnet_synth_fixed.json"
    path_to_save = "./datasets"

In [ ]:
if Config.previous_data is not None and os.path.isfile(Config.previous_data):
    with open(Config.previous_data, "r") as f:
        points = json.load(f)
else:
    with open(Config.input_dataset, "r") as f:
        points = json.load(f)
print(len(points))

## Manual fix


In [ ]:
def pretty_print_text(text: str, label: str, expandable: bool = False) -> None:
    lines = text.split("\n")
    wrapped_text = "\n".join(
        [line_part for line in lines for line_part in textwrap.wrap(str(line), 180, break_long_words=False)]
    )
    if expandable:
        display(
            widgets.Accordion(
                children=[
                    widgets.HTML(
                        value=f'<div style="font-family:Consolas;white-space:pre;line-height:18px">{wrapped_text.replace("&", "&amp;").replace("<", "&lt;")}</div>',
                        layout=widgets.Layout(width="auto", height="100%", max_height="100%"),
                        rows=10,
                    )
                ],
                titles=(label,),
            )
        )
    else:
        display(
            widgets.HTML(
                value=f'<div style="white-space:pre">{wrapped_text}</div>',
                description=f"{label}: ",
                layout=widgets.Layout(width="auto", height="100%"),
            )
        )


def display_plantuml(uml_code: str) -> None:
    pl = plantuml.PlantUML()
    image_data = pl.processes(uml_code)
    display(Image(image_data))

In [ ]:
q_to_labels = {
    "JSON Graph": "diagram",
    "QUERY": "query",
    "TEXT": "text_answer",
}

all_widgets = {}
for q in q_to_labels:
    widget_axis = tuple(
        [
            widgets.Textarea(
                description=q,
                value=points[i][q_to_labels[q]],
                layout=widgets.Layout(width="auto", height="100%", max_height="100%"),
                rows=4,
            )
            for i in range(len(points))
        ]
    )
    all_widgets[q] = widget_axis


def screen_cur_labels(save_with_name, all_widgets: Dict, points: List) -> None:
    for i in range(len(points)):
        for q, widget_axis in all_widgets.items():
            points[i][q_to_labels[q]] = widget_axis[i].value
    if save_with_name:
        filename = filename_.value
    else:
        filename = "chkpt_" + strftime("%Y-%m-%d", gmtime())
    save_dir = Config.path_to_save

    full_path = os.path.join(save_dir, f"{filename}.json")
    os.makedirs(save_dir, exist_ok=True)
    with open(full_path, "w") as file:
        json.dump(points, file, indent=2)


save_state = False
next_state = False
slider = widgets.IntSlider(min=0, max=len(points) - 1, step=1, value=0)


def get_studio(save_click: bool, point_n: int, next_click: bool, all_widgets: Dict, points: List) -> None:
    global save_state, next_state, slider
    if next_state != next_click:
        next_state = next_click
        slider.value += 1
    else:
        save_with_name = save_state != save_click
        save_state = save_click
        screen_cur_labels(save_with_name, all_widgets, points)
        for q, widget_axis in all_widgets.items():
            display(widget_axis[point_n])
        pretty_print_text(points[point_n]["path"], "PATH")
        pretty_print_text(points[point_n]["language"], "LANGUAGE")
        pretty_print_text(points[point_n]["version"], "VERSION")
        pretty_print_text(points[point_n]["code"], "CODE", expandable=True)
        diagram = points[point_n]["diagram"]
        converted_diagram, errors, _ = convert_str_graph_with_check(
            diagram, points[point_n]["code"], GraphConvertConfig()
        )
        if len(errors) > 0:
            pretty_print_text("\n".join([e.description for e in errors]), "ERRORS", expandable=True)
        if converted_diagram is not None:
            pretty_print_text(converted_diagram, "DIAGRAM", expandable=True)
            display_plantuml(converted_diagram)

In [ ]:
default_filename = "fixed_synth"
filename_ = widgets.Text(
    placeholder="filename to save",
    description="Filename:",
    value=default_filename,
)

In [ ]:
display(filename_)
print("\n")

interact(
    get_studio,
    save_click=widgets.ToggleButton(description="Click to Save", button_style="info", value=False),
    point_n=slider,
    next_click=widgets.ToggleButton(description="Next point", button_style="warning", value=False),
    all_widgets=fixed(all_widgets),
    points=fixed(points),
)

print()

## Final Conversion


In [ ]:
bad_points = {27, 36, 69, 70, 75, 76, 77, 133, 153, 154, 155, 267}
possible_versions_map = {
    28: [27, 28],
    37: [36, 37],
    71: [69, 70, 71],
    134: [133, 134],
    268: [267, 268],
}
final_points = []
for i, p in enumerate(points):
    if i in bad_points:
        continue
    diagram = p["diagram"]
    possible_version = [
        points[additional_point_i]["version"] for additional_point_i in possible_versions_map.get(i, [i])
    ]
    _, _, converted_graph = convert_str_graph_with_check(diagram, p["code"], GraphConvertConfig())
    final_points.append(p | {"diagram": str(converted_graph), "possible_version": possible_version})

with open(os.path.join(Config.path_to_save, f"{filename_.value}_final.json"), "w") as f:
    json.dump(final_points, f, indent=2)